In [55]:
import pdfplumber
import json
import re
from elasticsearch import Elasticsearch
from functools import partial
import pandas as pd


Config ignore table when extract_text

In [60]:
def not_within_bboxes(obj, bboxes):
    """Check if the object is in any of the table's bbox."""

    def obj_in_bbox(_bbox):
        """Define objects in box.

        See https://github.com/jsvine/pdfplumber/blob/stable/pdfplumber/table.py#L404
        """
        v_mid = (obj["top"] + obj["bottom"]) / 2
        h_mid = (obj["x0"] + obj["x1"]) / 2
        x0, top, x1, bottom = _bbox
        return (h_mid >= x0) and (h_mid < x1) and (v_mid >= top) and (v_mid < bottom)

    return not any(obj_in_bbox(__bbox) for __bbox in bboxes)

def filter_tables(page: pdfplumber.page.Page) -> pdfplumber.page.Page:
    if page.find_tables() != []:
        bboxes = [table.bbox for table in page.find_tables()]
        bbox_not_within_bboxes = partial(not_within_bboxes, bboxes=bboxes)
        page = page.filter(bbox_not_within_bboxes)

    return page
  


Create raw data after read from pdf (ignore table)

In [61]:

with pdfplumber.open('s3-userguide.pdf') as pdf:
    # init raw data array
    lines_data = []
    # loop each page with 
    for page_num, page in enumerate(pdf.pages, start=1):
        lines = filter_tables(page).extract_text(layout=True).split('\n')
        for line_num, line in enumerate(lines, start=1):
            lines_data.append({
                "content": line,
                "title": lines[2],
                "section_title": lines[-4],
            })

    with open('raw.json', 'w', encoding='utf-8') as f:
        json.dump(lines_data, f, ensure_ascii=False, indent=4)

Clean, add rule and group raw data to data which can import to ES

In [62]:
with open('raw.json', 'r', encoding='utf-8') as f:
    lines_data = json.load(f)

filtered_lines_data = []
table_of_contents = {}
current_content = ''
for line_data in lines_data:
    content = line_data['content'].strip()
    if content != '':
        content = re.sub(r'\s+', ' ', content)
        if ("........." in content):
            content = re.sub(r'\.+', '@@@', content)
            title = content.split("@@@")[0].strip()
            page_num = content.split("@@@")[1].strip()
            table_of_contents[title] = page_num
            continue
        if "API Version 2006-03-01" not in content and "Amazon Simple Storage Service" not in content and content != '':
            if current_content:
              if (content[0] == '-' or content[0] == '•' or content[0].isnumeric()):
                  current_content += '\n' + content
              elif content[0].islower():
                  current_content += ' ' + content
              elif (current_content[-1] != '.'):
                  current_content += ' ' + content
              elif  (current_content[-1] == ',' or current_content[-1] == ':'):
                  current_content += ' ' + content
              elif content[0].islower():
                  current_content += ' ' + content
              else:
                  new = {}
                  new['section_title'] = ''
                  page = line_data['section_title'].strip().split(" ")[-1]
                  new['title'] = re.sub(r'\s+', ' ', line_data['title'].strip())
                  new['content'] = current_content
                  if page in table_of_contents.values():
                      new['section_title'] = list(table_of_contents.keys())[list(table_of_contents.values()).index(page)]
                  else:
                      new['section_title'] = line_data['section_title'].strip().split("API Version")[0].strip()
                  
                  new['page'] = page
                  if new['section_title'] and "Table of Contents" not in new['content']:
                      filtered_lines_data.append(new)
                  current_content = content
            else:
                current_content = content
print(table_of_contents)           

with open('output.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_lines_data, f, ensure_ascii=False, indent=4)


{'What is Amazon S3?': '1', 'Features of Amazon S3': '1', 'Storage classes': '1', 'Storage management': '2', 'Access management and security': '469', 'Data processing': '4', 'Storage logging and monitoring': '2343', 'Analytics and insights': '5', 'Strong consistency': '2343', 'How Amazon S3 works': '5', 'Buckets': '2367', 'Objects': '2340', 'Keys': '2340', 'S3 Versioning': '2341', 'Version ID': '2341', 'Bucket policy': '2341', 'S3 Access Points': '8', 'Access control lists (ACLs)': '9', 'Regions': '2339', 'Amazon S3 data consistency model': '10', 'Concurrent applications': '11', 'Related services': '2344', 'Accessing Amazon S3': '13', 'AWS Management Console': '2344', 'AWS Command Line Interface': '2344', 'AWS SDKs': '2345', 'Amazon S3 REST API': '14', 'Paying for Amazon S3': '15', 'PCI DSS compliance': '15', 'Getting started': '34', 'Setting up': '17', 'Sign up for an AWS account': '17', 'Create a user with administrative access': '18', 'Step 1: Create a bucket': '135', 'Step 2: Uploa

Put data to ES

In [ ]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

with open('output.json', 'r') as file:
    data = json.load(file)

for doc in data:
    es.index(index='s3_guide', body=doc)

/var/folders/yz/tvpwz_9d6c56fms92tltys800000gn/T/ipykernel_55600/2731284087.py:8: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index='s3_guide', body=doc)
